In [ ]:
import math
import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np

In [ ]:
class Implicit(nn.Module):
  def __init__(self, fun, dimension):
    super(Implicit, self).__init__()

    d_in = dimension
    dims = [512, 512, 512, 512, 512, 512, 512, 512]
    beta = 100
    skip_in = [4]
    radius_init = 1
    dims = [d_in] + dims + [1]

    self.num_layers = len(dims)
    self.skip_in = skip_in

    for layer in range(0, self.num_layers - 1):
      if layer + 1 in skip_in:
        out_dim = dims[layer + 1] - d_in
      else:
        out_dim = dims[layer + 1]
      lin = nn.Linear(dims[layer], out_dim)
      if layer == self.num_layers - 2:
        torch.nn.init.normal_(lin.weight, mean=np.sqrt(np.pi) / np.sqrt(dims[layer]), std=0.00001)
        torch.nn.init.constant_(lin.bias, -radius_init)
      else:
        torch.nn.init.constant_(lin.bias, 0.0)
        torch.nn.init.normal_(lin.weight, 0.0, np.sqrt(2) / np.sqrt(out_dim))
      setattr(self, "lin" + str(layer), lin)
    self.activation = nn.Softplus(beta=beta)
    #self.activation = nn.ReLU()
    self.fun = fun

  def forward(self, input):
    x = input
    for layer in range(0, self.num_layers - 1):
      lin = getattr(self, "lin" + str(layer))
      if layer in self.skip_in:
        x = torch.cat([x, input], -1) / np.sqrt(2)
      x = lin(x)
      if layer < self.num_layers - 2:
        x = self.activation(x)
    fun_sign = torch.tanh(0.1*self.fun(input))
    fun_sign = fun_sign.reshape(x.shape)
    x = x * fun_sign
    return x

In [ ]:
def grad(y, x):
  g = autograd.grad(y, [x], grad_outputs=torch.ones_like(y), create_graph=True)[0]
  return g

def div(y, x):
  div = 0.0
  for i in range(y.shape[-1]):
    div += autograd.grad(y[..., i], x, grad_outputs=torch.ones_like(y[..., i]), create_graph=True)[0][..., i:i+1]
  return div

def Laplacian(y, x):
  g = grad(y, x)
  return div(g, x)

def pLaplacian(y, x, p=2):
  g = grad(y, x)
  #g_n = g.norm(2, dim=1)
  g_n = torch.linalg.norm(g, 2, dim=1)
  g_n = g_n**(p-2)
  g_n = torch.reshape(g_n, (g.shape[0],1))
  g = g_n * g
  return div(g, x)

In [ ]:
def uniformSamples(num_points, grid_min, grid_max, device):
  xx = torch.FloatTensor(num_points, 1).uniform_(grid_min[0], grid_max[0])
  yy = torch.FloatTensor(num_points, 1).uniform_(grid_min[1], grid_max[1])
  zz = torch.FloatTensor(num_points, 1).uniform_(grid_min[2], grid_max[2])
  x = torch.cat((xx,yy,zz), dim=-1)
  return x.to(device)

In [ ]:
def trainPPoisson(num_iters, fun, grid_min, grid_max, p=2, device='cpu'):
  assert(len(grid_min) == len(grid_max))
  dimension = len(grid_min)

  model = Implicit(fun=fun, dimension=dimension).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

  n_samples = 1024
  loss = 0

  # regular Ggrid data
  #res = (16,16,16)
  #x = grid_samples(res, grid_min, grid_max, device)
  #x.requires_grad = True

  # Train network
  for i in range(0, num_iters):
    # Uniform samples
    x = uniformSamples(n_samples, grid_min, grid_max, device)
    x.requires_grad = True

    # Input implicit 
    fun_d = fun(x)

    model.train()
    optimizer.zero_grad()
    f_d = model(x)

    # Laplacian 
    #lap = laplacian(f_d, coords_d)
    #loss = torch.mean((lap + 1)**2)
    # p-Laplacian
    lap = pLaplacian(f_d, x, p)
    lap_constraint = torch.mean((lap+1)**2)

    # Penalize extra zeros
    extra_constraint = torch.mean(torch.where(torch.abs(fun_d)<1e-3, torch.zeros_like(f_d), torch.exp(-1e2 * torch.abs(f_d))))

    loss = lap_constraint + extra_constraint

    #if i%500 == 0:
    #print(loss)

    loss.backward()
    #clip_grad = 1.0
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_grad)
    optimizer.step()

  # model.eval()
  return model

In [ ]:
def trainEikonal(num_iters, fun, grid_min, grid_max, device='cpu'):
  assert(len(grid_min) == len(grid_max))
  dimension = len(grid_min)

  model = Implicit(fun=fun, dimension=dimension).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

  n_samples = 1024
  loss = 0

  # regular Ggrid data
  #res = (16,16,16)
  #x = grid_samples(res, grid_min, grid_max, device)
  #x.requires_grad = True

  # Train network
  for i in range(0, num_iters):
    # Uniform samples
    x = uniformSamples(n_samples, grid_min, grid_max, device)
    x.requires_grad = True

    # Input implicit 
    fun_d = fun(x)

    model.train()
    optimizer.zero_grad()
    f_d = model(x)

    g_d = grad(f_d, x)
    g_norm = (g_d.norm(2, dim=1) - 1)**2
    g_constraint = torch.mean(g_norm)

    # Penalize extra zeros
    extra_constraint = torch.mean(torch.where(torch.abs(fun_d)<1e-3, torch.zeros_like(f_d), torch.exp(-1e2 * torch.abs(f_d))))

    loss = g_constraint + extra_constraint

    #if i%500 == 0:
    #print(loss)

    loss.backward()
    #clip_grad = 1.0
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_grad)
    optimizer.step()

  # model.eval()
  return model

In [ ]:
def saveVTK(filename, xyz, res, field):
  resx, resy, resz = res
    
  field_title = 'VALUE'

  with open(filename, 'w') as f:
    f.write('# vtk DataFile Version 3.0\n')
    f.write('vtk output\n')
    f.write('ASCII\n')
    f.write('DATASET STRUCTURED_GRID\n')
    f.write('DIMENSIONS ' + str(resx) + ' ' + str(resy) + ' ' + str(resz) +'\n')
    f.write('POINTS ' + str(resx*resy*resz) + ' double\n')

    if (torch.is_tensor(xyz)):
      np.savetxt(f, xyz.detach().cpu().numpy())
    else:
      np.savetxt(f, xyz)
    
    f.write('\n\n')

    f.write('POINT_DATA ' + str(resx*resy*resz) + '\n')
    f.write('SCALARS ' + field_title + ' double' + '\n')
    f.write('LOOKUP_TABLE default\n')
        
    if (torch.is_tensor(field)):
      np.savetxt(f, field.detach().cpu().numpy())
    else:
      np.savetxt(f, field)
    f.write('\n')

In [ ]:
def torchLinearGrid(grid_min, grid_max, grid_res, device='cpu'):
  resx, resy, resz = grid_res
  dx = grid_max[0]-grid_min[0]
  x = torch.arange(grid_min[0], grid_max[0], step=dx/float(resx))
  dy = grid_max[1]-grid_min[1]
  y = torch.arange(grid_min[1], grid_max[1], step=dy/float(resy))
  dz = grid_max[2]-grid_min[2]
  z = torch.arange(grid_min[2], grid_max[2], step=dz/float(resz))
  xx, yy, zz = torch.meshgrid(x, y, z)
  xx = xx.to(device)
  yy = yy.to(device)
  zz = zz.to(device)
  dimg = resx * resy * resz
  xyz = torch.stack((xx, yy, zz), dim=-1).reshape(dimg,3)
  return xyz

In [ ]:
def torchLinearSampling(model, xyz):
  d = model(xyz)
  return d

In [ ]:
# Primitives and operations
def sphere(p, center, r):
  x0 = p[:,0] - center[0]
  x1 = p[:,1] - center[1]
  x2 = p[:,2] - center[2]
  return r**2 - x0**2 - x1**2 - x2**2

def cylX(p, center, r):
  x1 = p[:,1] - center[1]
  x2 = p[:,2] - center[2]
  return r**2 - x1**2 - x2**2

def cylY(p, center, r):
  x0 = p[:,0] - center[0]
  x2 = p[:,2] - center[2]
  return r**2 - x0**2 - x2**2

def cylZ(p, center, r):
  x0 = p[:,0] - center[0]
  x1 = p[:,1] - center[1]
  return r**2 - x0**2 - x1**2

def block(p, vertex, dx, dy, dz):
  x0 = -(p[:,0]-vertex[0]) * (p[:,0]-(vertex[0]+dx))
  x1 = -(p[:,1]-vertex[1]) * (p[:,1]-(vertex[1]+dy))
  x2 = -(p[:,2]-vertex[2]) * (p[:,2]-(vertex[2]+dz))
  t0 = x0 + x1 - torch.sqrt(x0**2 + x1**2)
  return t0 + x2 - torch.sqrt(t0**2 + x2**2)

def difference(d1, d2):
  d = d1 - d2 - torch.sqrt(d1**2 + d2**2)
  return d

def intersection(d1, d2):
  d = d1 + d2 - torch.sqrt(d1**2 + d2**2)
  return d

In [ ]:
def model(p):
  sp1 = sphere(p, center=(0,0,0), r=1)
  b1 = block(p, vertex=(-0.75, -0.75, -0.75), dx=1.5, dy=1.5, dz=1.5)
  t1 = intersection(sp1, b1)
  c1 = cylX(p, center=(0,0,0), r=0.5)
  c2 = cylY(p, center=(0,0,0), r=0.5)
  c3 = cylZ(p, center=(0,0,0), r=0.5)
  t2 = difference(t1, c1)
  t3 = difference(t2, c2)
  t4 = difference(t3, c3)
  return t4

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
xyz = torchLinearGrid(grid_min=(-1.5,-1.5,-1.5), grid_max=(1.5,1.5,1.5), grid_res=(64,64,64), device=device)

In [ ]:
# Original model
f_orig_xyz = model(xyz)
saveVTK('3d_model_orig.vtk', xyz, (64,64,64), f_orig_xyz)

In [ ]:
# p-Poisson
#model_p8 = trainPPoisson(num_iters=10000, fun=model, grid_min=(-1.5,-1.5,-1.5), grid_max=(1.5,1.5,1.5), p=8, device=device)

In [ ]:
#f_p8_xyz = torchLinearSampling(model_p8, xyz)
#saveVTK('3d_model_pPoisson_p8.vtk', xyz, (64,64,64), f_p8_xyz)

In [ ]:
# Eikonal
model_eik = trainEikonal(num_iters=10000, fun=model, grid_min=(-1.5,-1.5,-1.5), grid_max=(1.5,1.5,1.5), device=device)

In [ ]:
f_eik_xyz = torchLinearSampling(model_eik, xyz)
saveVTK('3d_model_eikonal.vtk', xyz, (64,64,64), f_eik_xyz)